---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
#df.head(10)

In [6]:
def date_sorter():
    
    # Load the data
    import pandas as pd

    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)
    
    # Match the various Date formats
    tmp1 = df.str.extractall(r'(?P<date>(?P<month>\d{1,2})[-/](?P<day>\d{1,2})[-/](?P<year>\d{2,4}))')
    tmp2 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,\.]{1,2}(?P<day>\d{1,2})[ ,\.]{1,2}(?P<year>\d{2,4}))')
    tmp3 = df.str.extractall(r'(?P<date>(?P<day>\d{2})[ ,\.-]+(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ , \.-]+(?P<year>\d{2,4}))')
    tmp4 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?P<day>\d{1,2})[thsnd]{2}, (?P<year>\d{4}))')
    tmp5 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,](?P<year>\d{4}))')
    tmp6 = df.str.extractall(r'(?P<date>(?P<month>\d{1,2})/(?P<year>\d{4}))')
    tmp7 = df.str.extractall(r'(?P<date>(?P<year>[ ]*[1-2]\d{3}[ ]*))')
    
    # map the months into integers
    months_label = {'Jan': '01',
                    'Feb': '02',
                    'Mar': '03',
                    'Apr': '04',
                    'May': '05',
                    'Jun': '06',
                    'Jul': '07',
                    'Aug': '08',
                    'Sep': '09',
                    'Oct': '10',
                    'Nov': '11',
                    'Dec': '12'
    }
    tmp2['month'] = tmp2['month'].map(months_label)
    tmp3['month'] = tmp3['month'].map(months_label)
    tmp4['month'] = tmp4['month'].map(months_label)
    tmp5['month'] = tmp5['month'].map(months_label)
    
    # Add the missing columns to some of the data.frames
    tmp5['day'] = tmp6['day'] = tmp7['day'] = ''
    tmp7['month'] = ''
    
    # Concatenate the data.frames
    tmplist = [tmp2, tmp3, tmp4, tmp5, tmp6, tmp7]
    final = tmp1
    for tmp in tmplist:
        final = pd.concat([final, tmp[['date', 'year', 'month', 'day']]], axis = 0)
        
    # Fill gaps
    final.replace('', '01', inplace=True)
    final.fillna(value='01', inplace=True)
    
    # Pad month, day, and year to get the proper string lengths
    final['day'] = final['day'].str.zfill(2)
    final['month'] = final['month'].str.zfill(2)
    final['year'] = final['year'].str.rjust(width = 3, fillchar = '9').str.rjust(width = 4, fillchar = '1')
    
    # Let's prepare the output
    final_out = (final['year']+final['month']+final['day']).str.replace(' ', '').astype(float)
    final_out.index = final_out.index.get_level_values(0)
    
    tmp_out = final_out.reset_index(level=[0]).groupby('index').max()
    tmp_out.columns = ['date']
    tmp_out.sort_values('date', inplace=True)
    
    # for debugging purposes
    final_df = tmp_out.merge(df.to_frame(), left_index = True, right_index = True).reset_index(level=[0])
    return final_df#final_df['index'].rename(None)

In [7]:
out = date_sorter()

/Applications/anaconda3/envs/testingenv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [8]:
out

,index,date,0
0,9,19710410.0,(4/10/71)Score-1Audit C Score Current:\n
1,84,19710518.0,5/18/71 Total time of visit (in minutes):\n
2,2,19710708.0,sshe plans to move as of 7/8/71 In-Home Servic...
3,53,19710711.0,7/11/71 SOS-10 Total Score:\n
4,28,19710912.0,9/12/71 [report_end]\n
5,474,19720101.0,sPatient reported losing three friends that pa...
6,153,19720113.0,13 Jan 1972 Primary Care Doctor:\n
7,13,19720126.0,1/26/72 Communication with referring physician...
8,129,19720506.0,06 May 1972 SOS-10 Total Score:\n
9,98,19720513.0,5/13/72 Other Adult Mental Health Outcomes Sca...


In [9]:
out.to_csv('assignment1_out.csv')

In [13]:
out.iloc[5, 2]

'sPatient reported losing three friends that passed away during his deployment, including a close friend Jacques, and two other friends that he lived with for a period of time. A fourth friend passed away prior to his deployment in 1972.  Patient reported thinking about his friends daily and described his feelings of grief as remaining constant since they died.  Alcohol Use: How often did you have a drink containing alcohol in the past year: Two to four times a month (2 points)\n'

In [16]:
out.iloc[44, 2]

'4/11/1974 Chief Complaint / HPI Chief Complaint (Patients own words)\n'